**Разбивка на подзадачи**:
- подготовка данных
- методика расчета средних охватов по станции (15мин, день, неделя, месяц)
- методика расчета суммарного рейтинга GRP
- методика расчета Reach 1+ для однодневной рекламной кампании (РК)  
- методика Reach 1+ для расчета РК по нескольким дням внутри одной недели (пересечение однодневных охватов)
- методика расчета частотного охвата

In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib
import seaborn as sns
import scipy as sc
import functions
import warnings
warnings.filterwarnings ( action='ignore')

In [2]:
# 1. Подготовка данных

Исходные данные представлены в виде 2 таблиц: в одной закодированы дневники слушания по каждому дню недели в разрезе 15 минуток, во второй закодированы соц-дем профиль, в том числе ответы на вопросы о частоте слушания той или иной радиостанции.  
Исследование проводилось в г. Москва в 1 кв 2021г методом телефонного опроса Day-After-Recall, и включает результаты опроса 15 138 респондентов.   
Таблицу с дневниками слушания загрузим полностью, таблица с соц-демом содержит 500 с лишним колонок, поэтому для расчетов возьмем выборку из этой таблицы, которая содержит данные с ответами на вопросы о частоте слушания Авторадио.



In [3]:
# Дневники слушания (сведенные в одну таблицу)
diary = pd.read_csv ('./src/data/diary.csv', sep=';')
diary

,Member_nr,day,Chl_id,weights,1,2,3,4,5,6,...,87,88,89,90,91,92,93,94,95,96
0,1000000001,3,152,0.6037,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000000002,5,112,1.1176,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000000004,7,115,1.0779,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000000004,7,199,1.0779,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000000005,5,101,1.0603,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26403,1000015138,4,127,0.7903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26404,1000015138,4,164,0.7903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26405,1000015138,4,170,0.7903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26406,1000015138,4,179,0.7903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Описание колонок:  
**Member_nr** - уникальный ID респондента  
**day** - закодированный день недели слушания  
**Chl_id** - код радиостанции   
**weights** - веса респондентов (в методиках часто используется другое обозначение *projection factor*), это количество людей в ген. совокупности соотвествующее одной весовой единице в опросе. Проще говоря, это поправочный коэфициент, который позволяет выровнять распредление основных соц-дем характеристик (пол, возраст, размер семьи) между выборкой и ген. совокупностью.      
**1...96** - номера слотов (15 минутных интервалов в сутках), в которых указан рейтинг слушания:  
- ноль, если респондент не слушал ни одной радиостанции в данном 15-мин интервале
- единица деленная на количество станций, которые слушал респондент в данном 15-мин интервале, в противном случае    

In [4]:
# Выборка из соц-дем таблицы
rt = pd.read_csv ('./src/data/data.csv', sep=';')
rt

,ID,w,day_list,week_list,month_list
0,1000000001,0.6037,0,0,0
1,1000000002,1.1176,0,0,0
2,1000000003,0.8871,0,0,0
3,1000000004,1.0779,0,0,0
4,1000000005,1.0603,1,7,1
...,...,...,...,...,...
15133,1000015134,1.7235,1,7,1
15134,1000015135,0.8765,0,6,1
15135,1000015136,0.8866,0,0,0
15136,1000015137,0.8601,0,0,1


Описание колонок:  
**ID** - уникальный ID респондента  
**w** - веса респондентов  
**day_list** - индикатор суточного слушания "Авторадио"   
**week_list** - индикатор неедельного слушания "Авторадио" (количество дней недели)   
**month_list** - индикатор месячного слушания "Авторадио" 

В качестве бенчмарка используем 3 медиаплана, рассчитанные в SuperNova MediaScope по следующим вводным:    
**База данных:** Radio Index - Moscow. January - March 2021  
**Размер генеральной совокупности (тыс.):** 10 973,3  
**Размер целевой группы (тыс.):** 10 973,3  
**Выборка:** 15 138    
**Целевые медиа:** Авторадио  
**МП1** - 2 выхода в час, 4 выхода в день, с 10 до 11, с 12 до 13, с 14 до 15, с 18 до 19 часов, период: понедельник (одна неделя), итого - 8 выходов  
**МП2** - 2 выхода в час, 4 выхода в день, с 10 до 11, с 12 до 13, с 14 до 15, с 18 до 19 часов, период: понедельник-пятница  (одна неделя), итого - 40 выходов  
**МП3** - 4 выхода в час, 3 выхода в день, с 15 до 18 часов, период: понедельник-воскресенье  (четыре недели), итого - 336 выходов  

In [5]:
mp1 = pd.read_csv ('./src/data/mp1.csv', sep=';').dropna()
mp2 = pd.read_csv ('./src/data/mp2.csv', sep=';').dropna()
mp3 = pd.read_csv ('./src/data/mp3.csv', sep=';').dropna()
mp1

,Показатель,Авторадио
0,GI,555.88
1,Frequency,1.82
2,Index T/U Reach,100.00
3,GRP,5.07
4,TRP,5.07
5,RP Index,100.00
6,Spots,8.00
7,Reach 1+,306.13
8,Reach 2+,120.83
9,Reach 3+,58.34


На последнем этапе подготовки зафиксируем переменные среды, так как вызовы функций настроены на входные данные, которые должны содержать разные таргетинги, разные станции и разное расписание выходов, а мы работаем с зафиксированными вводными, чтобы каждый раз не передавать эти данные, запишем их в константы

In [6]:
CH = 101 # код Авторадио
#AUD = rt['ID'].to_list() # ID целевой аудитории в виде списка, в нашем случае это все ID
MP1_D = [1] # номер дня недели выхода ролика
MP2_D = [1,2,3,4,5]
MP3_D = [1,2,3,4,5,6,7]
MP1_SCH = [10,12,14,18] # часы выходов
MP2_SCH = MP1_SCH
MP3_SCH = [15,16,17]
MP1_N = 2 # количество выходов в час
MP2_N = MP1_N
MP3_N = 4

In [7]:
# 2.  Подсчет аудиторий станции (15 мин, день, неделя, месяц)

Расчет рейтинга 15 минутки идет по формуле:
$\sum_{i = 1}^{n}\dfrac{w_i*t_i}{Day\_Aud}$, где 
- $n$ - количество респондентов, имеюших записи в дневнике слушания в выбранный день 
- $w_i$ веса всех респондентов, присутствующих в дневнике слушания в выбранный день
- $t_i$ рейтинг слушания выбранной радиостанции в интервале 15 минут (значение от нуля до единцы, в зависимости от количества слушаемых станций) 
- $Day\_Aud$ сумма весов респондентов, опрошенных в выбранный день  
Последнее значение в расчетах можно принять за константу равную $\frac{15318}{7}$, так как волна опроса соотвествующим образом была равномерно распределена по дням недели

In [8]:
# Итоговая функция для подсчета рейтинга 15-минутки
# считаем на примере: понедельник, Авторадио, интервал: 14:00 - 14:15 
# нумерация 15-минуток в дневниках слушания начинается с 5:00 утра, так что времени 14:00 соответствует 37 слот
round (functions.reach_slot(MP1_D[0], CH ,'37')*100, 2)

0.91

Теперь подсчитаем показатель AQH (средний рейтинг 15-минутки) для Авторадио, по методике для этого надо рассчитать рейтинги всех 15 минуток по всем 7 дням недели и усреднить их.  
При этом в отчете MediaScope за 1 кв 2021г, значения AQH даны для диапазона 06:00-24:00, поэтому нам также нужны значения только для этого интервала времени 

In [9]:
# AQH Авторадио
res=[]
for day in range (1,8):
    for slot in range (5, 77):
        res.append (functions.reach_slot ( day, CH, str(slot)))
print ('AQH Авторадио -', round (np.array(res).mean() ,3) *100) 

AQH Авторадио - 0.5


Сравниваем с бенчмарком - все совпало
<img src='./src/images/AQH.png'>

Теперь перейдем к расчету средних рейтингов: **Reach Daily**, **Reach Weekly**, **Reach Monthly**  
Они определяются очень просто - как доля положительно ответивших на соответствующий вопрос (с учетом весов) по отношению к общему количеству опрошенных.   
Получим соотвествующие значения для Авторадио

In [10]:
print ('Средний дневной охват "Авторадио", Daily Reach-', round(np.dot(rt['day_list'],rt['w'])/15138 *100, 2 )) 
print ('Средний недельный охват "Авторадио", Weekly Reach -', round(np.dot(np.where (rt['week_list']==0,0,1),rt['w'])/15138*100,1))
print ('Средний месячный охват "Авторадио", Monthly Reach -', round(np.dot(rt['month_list'],rt['w'])/15138 *100, 2 )) 

Средний дневной охват "Авторадио", Daily Reach- 9.1
Средний недельный охват "Авторадио", Weekly Reach - 25.2
Средний месячный охват "Авторадио", Monthly Reach - 34.55


Расчеты верны, можем смело опираться на эти формулы в дальнейшем

In [11]:
# 3. Расчет GRP

Переходим к расчетам медиапоказателей РК, начнем с самого простого: подсчету GRP/TRP.  
Итоговая формула для расчета GRP выглядит следующим образом:  
$GRP = \sum_{d}\sum_{i = 1}^{k} Reach\_slot_i*\dfrac{n_i}{4}$, где
- $d$ - дни выходов ролика в течении РК
- $k$ - количество слотов в которых есть выходы ролика
- $Reach\_slot$ - рейтинг 15-минутки
- $n$ - количество выходов ролика в час  
По сути это просто сумма рейтингов 15-минуток, с поправкой на количество выходов в час. При медиапланировании РК на радио минимальным периодом планирования является час и ролики равномерно распределяются по сетке выходов, соответственно, если количество выходов в час равно 1, то рассматривая эту ситуацию с точки зрения теории вероятности, мы понимаем что, с вероятностью 0,25 выход ролика попадет в любой из 4 слотов, итоговую вероятность по каждому респонденту надо оценивать как 
сумму рейтингов слушания 15 минуток с коэфициентом 0,25. Если будет 2 выхода в час, то 1 ролик выйдет в первые полчаса (первые 2 слота часа), а ворой ролик во вторые полчаса (3,4 слот часа). Соответственно, первый ролик с вероятностью 0,5 попадет в 1 слот, либо с вероятностью 0,5 во второй. Аналогично со вторым роликом и вероятностями попасть в 3 и 4 слоты. В итоге получается формула для расчета поправочного коэфициента $\dfrac{n}{4}$  

*Примечание*: поскольку мы изначально зафиксировали в качестве целевой аудитории всех респондентов, в нашем случае GRP=TRP.  На практике чаще всего работают с целевой аудиторией и TRP рассчитывается отдельно. Формула расчета аналогична формуле расчета GRP, только рейтинги 15-минуток считаются только по целевой аудитории (то есть в числителе суммируются только веса респондентов из целевой аудитории, а в знаменателе в сумме задействованы только веса респондентов, принадлежащих целевой аудитории).  
Для демонстрации работы используется функция-обертка основной функции для расчета GRP/TRP

Посчитаем GRP по формуле и сравним с данными из медиапланов

In [12]:
print ("Расчетное значение GRP: ", functions.grp (CH, MP1_D, MP1_SCH, MP1_N) , ' | Значение МП 1: ' , mp1.to_dict (orient='records')[3])
print ("Расчетное значение GRP: ", functions.grp (CH, MP2_D, MP2_SCH, MP2_N) , '| Значение МП 2: ' , mp2.to_dict (orient='records')[3])
print ("Расчетное значение GRP: ", functions.grp (CH, MP3_D, MP3_SCH, MP3_N) , '| Значение МП 3: ' , mp3.to_dict (orient='records')[3] )

Расчетное значение GRP:  5.07  | Значение МП 1:  {'Показатель': 'GRP', 'Авторадио': 5.07}
Расчетное значение GRP:  25.95 | Значение МП 2:  {'Показатель': 'GRP', 'Авторадио': 25.95}
Расчетное значение GRP:  52.86 | Значение МП 3:  {'Показатель ': 'GRP', 'Авторадио': '211.44'}


Данные совпали, кроме последнего медиаплана, так как последний МП рассчитан на 4 недели, а GRP считается только в рамках одной недели. Для получения корректной цифры надо отдельно высчитать GRP по каждой неделе размещения и затем просуммировать их. Так как у нас показатели РК не меняются от недели к неделе, то GRP по каждой неделе размещения идентичны и мы можем просто умножить GRP первой недели на 4.

In [13]:
print ("Расчетное значение GRP: ", functions.grp (CH, MP3_D, MP3_SCH, MP3_N)*4 , '| Значение МП 3: ' , mp3.to_dict (orient='records')[3] )

Расчетное значение GRP:  211.44 | Значение МП 3:  {'Показатель ': 'GRP', 'Авторадио': '211.44'}


In [14]:
mp3

,Показатель,Авторадио
0,GI,23203.1
1,Frequency,9.5
2,Index T/U Reach,100
3,GRP,211.44
4,TRP,211.44
5,RP Index,100
6,Spots,336
7,Reach 1+,2 444.47
8,Reach 2+,1 830.13
9,Reach 3+,1 511.29


functions.WR_cum (AUD, CH, MP1_D, MP1_SCH, MP1_N)

# 4. Расчет Reach 1+ 

Теперь переходим к охватам РК. Для начала рассмотрим методику расчета охвата в конкретный день размещения.
В методике охват рассматривается с точки зрения теории вероятностей, то есть для каждого респондента определяется вероятность услышать выход ролика в конкретную 15-минутку в виде $p_{ik} = r_{ik}*\dfrac{n}{4}$ , где $r_{ik}$ - рейтинг слушания респондента $i$ выбранной радиостанции в $k$ 15-минутку, $n$ - количество выходов ролика в час  
Далее все эти вероятности по респонденту $i$ суммируются по формуле:
$E_i = \sum_{s}\sum_{k = 1}^{m} p_{ik}$, где $m$ это количество 15-минуток размещения в течении дня, $s$ - радиостанция  
В итоге получается суммарная вероятность услышать ролик хотя бы 1 раз, если значение получилось больше 1, то мы принимаем $E_i=1$, затем по каждому респонденту мы считаем сумму произведений весов на показатель $E_i$ и делим на сумму весов респондентов, опрошенных в выбранный день:  
$Reach_{day}\space 1+= \sum_{i = 1}^{n}\dfrac{w_i*E_i}{Day\_Aud}$

*Примечание*: с точки зрения теории вероятностей надо учитывать совместную вероятность наступления события, и корректнее расчет производить по формуле единица минус вероятность события "респондент ни разу не услышал ролик" $p_0 = 1-((1-p_{ik_1})*(1-p_{ik_2})...(1-p_{ik_m}))$, но на практике количество множителей в последней формуле достаточно большое, для того, чтобы итоговое произведение было близко к нулю, поэтому я предполагаю, что для упрощения расчетов и используется простое суммирование вероятностей.

Проверим корректность формулы на примере первого медиаплана

In [38]:
print ("Расчетное значение Reach 1+: ", round (functions.Reach_day (CH, MP1_D[0], MP1_SCH, MP1_N)*100, 3),
       '| Значение из МП 1: ' , mp1.to_dict (orient='records')[12] )

Расчетное значение Reach 1+:  2.785 | Значение из МП 1:  {'Показатель': 'Reach%  1+', 'Авторадио': 2.79}


Формула работает, теперь более сложная задача посчитать охват на частоте 1+ для нескольких дней. По науке мы должны сложить охваты по всем дням и вычесть пересечения аудиторий. Но у нас нет данных по пересечению аудиторий, так в каждый день недели опрашивается отдельная группа респондентов и дневник слушания ограничен только этим днем. Поэтому нужно использовать формулу вероятности совместных событий. У нас есть данные по недельному охвату радиостанции, используя его в качестве базы для расчетов получаем следующую формулу:  
$RCR\ =WR*(1-(1-\dfrac{Reach_{d1}}{WR})\ast(1-\dfrac{Reach_{d2}}{WR})\ast(1-\dfrac{Reach_{d3}}{WR}).....\ast(1-\dfrac{Reach_{d7}}{WR}))$, где  
- $RCR$ - охват РК за несколько дней    
- $Reach_{dn}$ - охват РК в конкретный день $n$ (*Reach 1 +*)      
- $WR$ - недельный охват станции (*Weekly Reach*)

Попробуем посчитать охват для второго медиаплана, в котором период проведения РК: понедельник - пятница

In [75]:
# получаем недельный охват
WR = np.dot(np.where (rt['week_list']==0,0,1),rt['w'])/15138 
res=[]
#вычисляем RCR
for day in MP2_D:
    res.append(1-functions.Reach_day (CH, day, MP2_SCH, MP2_N)/WR) 
RCR = (1-np.array(res).prod())*WR*100
print ('Расчетное значение Reach 1+: ' ,  round (RCR, 2),
       '| Значение из МП 1: ' , mp2.to_dict (orient='records')[12] 
      ) 


Расчетное значение Reach 1+:  12.15 | Значение из МП 1:  {'Показатель': 'Reach%  1+', 'Авторадио': 12.27}


Рсчетный охват меньше ожидаемого (хоть и в пределах 5% интервала допустимого отклонения). Значит применяется каой-то повышающий коэфициент, для понимания нужно рассмотреть extreme case: что если по каждому неделю РК будет иметь максимальное возможное количество выходов во всех слотах на протяжении всего дня? По логике это будет означать, что мы охватим всех возможных слушателей этой радиостанции за неделю и охват РК будет равен недельному охвату станции.  Однако, если мы рассчитаем на таких вводных медиаплан по нашей формуле, мы увидим, что расчетные цифры будут меньше

In [ ]:
mp_full_day = list (np.arange(1,8))
mp_full_sch = list (np.arange (0,24))
WR = np.dot(np.where (rt['week_list']==0,0,1),rt['w'])/15138 
res=[]
#вычисляем RCR
for day in mp_full_day:
    res.append(1-functions.Reach_day (CH, day, mp_full_sch, 60)/WR) 
RCR = (1-np.array(res).prod())*WR*100
print ('Расчетное значение Reach 1+' , round (RCR,2), 'Weekly Reach', round (WR*100, 2))

In [ ]:
WR/RCR*100

Для корректировки значения и нужен повыщающий коэфицент, самое простое рассчитать поправочный коэфициент как $\dfrac{WR}{RCR_{max}}$ (в нашем случае это 1.0428) и домножать на него полученные расчетные значения RCR, но тогда мы столкнемся с другими трудностями: на малых значениях формула будет отрабатывать некорректно. Предположим, что мы в первый день охватили 1% от недельной аудитории и во второй день охватили 1% аудитории, получаем RCR равный **1.99%** от WR, однако домножив его на 1.0428 мы получим охват больше, чем **2%**, то есть больше максимально возможного значения.  
Соответственно, повыщающий коэфициент должен варьироваться в зависимости от того насколько значение RCR близко к максимуму.
Опытным путем была получена следующая формула:  
$RCF\ =\ (1-\dfrac{1}{RSR})\ast(\dfrac{TCR}{TSR})+1$

In [104]:
WR/RCR*100

1.0427843250020303

Для корректировки значения и нужен повыщающий коэфицент, самое простое рассчитать поправочный коэфициент как $\dfrac{WR}{RCR_{max}}$ (в нашем случае это 1.0428) и домножать на него полученные расчетные значения RCR, но тогда мы столкнемся с другими трудностями: на малых значениях формула будет отрабатывать некорректно. Предположим, что мы в первый день охватили 1% от недельной аудитории и во второй день охватили 1% аудитории, получаем RCR равный **1.99%** от WR, однако домножив его на 1.0428 мы получим охват больше, чем **2%**, то есть больше максимально возможного значения.  
Соответственно, повыщающий коэфициент должен варьироваться в зависимости от того насколько значение RCR близко к максимуму.
Опытным путем была получена следующая формула:  
$RCF\ =\ (1-\dfrac{1}{RSR})\ast(\dfrac{TCR}{TSR})+1$

In [50]:
mp2

,Показатель,Авторадио
0,GI,2847.30
1,Frequency,2.10
2,Index T/U Reach,100.00
3,GRP,25.95
4,TRP,25.95
5,RP Index,100.00
6,Spots,40.00
7,Reach 1+,1346.52
8,Reach 2+,624.68
9,Reach 3+,340.99


In [ ]:
Reach (n) = Reach(max) (1-(1-R/ Reach(max))n)

In [ ]:
aud = diary[(diary['Chl_id']==CH) & (diary['day']==day)]
col_list = [str(x) for x in spot_list(n_hours)]
aud['sum'] = aud.loc[:, col_list].sum(axis=1) * n / 4
aud['E'] = aud['sum'].apply(cut)
print () np.dot(aud['weights'], aud['E']) * 7 / 15138)

In [44]:
diary[diary['1']!=0]

,Member_nr,day,Chl_id,weights,1,2,3,4,5,6,...,87,88,89,90,91,92,93,94,95,96
174,1000000110,2,126,0.9289,0.5,0.000000,0.000000,0.25,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176,1000000110,2,160,0.9289,0.5,0.333333,0.333333,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,1000000125,5,101,0.7404,1.0,1.000000,1.000000,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217,1000000132,1,122,1.0096,1.0,1.000000,1.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
418,1000000220,4,109,0.6396,0.2,0.166667,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26070,1000014953,5,156,0.9138,1.0,1.000000,1.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26210,1000015012,3,123,1.0288,1.0,1.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26215,1000015013,5,128,1.1492,1.0,0.000000,0.000000,0.00,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26335,1000015089,1,114,0.9019,1.0,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Показатель,Авторадио
0,GI,555.88
1,Frequency,1.82
2,Index T/U Reach,100.00
3,GRP,5.07
4,TRP,5.07
5,RP Index,100.00
6,Spots,8.00
7,Reach 1+,306.13
8,Reach 2+,120.83
9,Reach 3+,58.34


In [ ]:
id_list, fin_aud, d, n, n_hours

In [36]:
1-((1-1/5)**16)

0.9718525023289344

In [34]:
0.8**5

0.3276800000000001

In [15]:
functions.WR_cum (AUD, CH, MP3_D, MP3_SCH, MP3_N)

NameError: name 'AUD' is not defined